In [400]:
import pandas as pd
import numpy as np
import math
import seaborn as sns 
import re
import ast

Задача проекта: разработать сервис для предсказания стоимости домов на основе истории предложений.
Так как нам необходимо предсказать числовое значение, то нам необходимо решить задачу регрессии.

Для начала изучим исходные данные.

In [401]:
df = pd.read_csv('data/data.csv')
df.head()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


В полученном датасете имеем следующие поля: \

status - текущий статус собственности \
private pool - наличие собственного бассейна \
property type - тип собственности \
street - улица и номер дома \
baths - количество ванных комнат \
homeFacts - дополнительная информация о собственности \
fireplace - информация о камине \
city - название города \
schools - информация о близлежащих школах \
sqft - площадь собственности \
zipcode - почтовый индекс \
beds - количество спальных мест \
state - штат \
stories - количество этажей \
mls-id - идентификационный номер члена MLS \
PrivatePool - наличие собственного бассейна \
MlsId - идентификационный номер члена MLS \
target - стоимость собственности

In [402]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [403]:
# удалим полные дубликаты строк
df = df.drop_duplicates()

В датасете достаточно много пропусков. Некоторые столбцы дублируют друг друга. \
Сначала проведем преобразование строк и выделим новые признаки.

##### Преобразуем столбец с целевым показателем.

In [404]:
# удаляем строки в которых целевой показатель отсутствует
df = df.drop(df[df['target'].isna()].index)

In [405]:
# найдем все строки в которых целевой показатель не является числом
df[~df['target'].str.isnumeric()]['target']

0           $418,000
1           $310,000
2         $2,895,000
3         $2,395,000
4             $5,000
             ...    
377180    $1,249,000
377181      $674,999
377182      $528,000
377183       $34,500
377184      $204,900
Name: target, Length: 374642, dtype: object

In [406]:
# уберем знак долара и запятую из целевого показателя для того, чтобы можно было его
# преобразовать
df['target'] = df['target'].apply(lambda target: target.replace('$', '')\
                                                       .replace(',', ''))

In [407]:
# еще раз проверим остались ли строки не являющиеся числом в столбце
df[~df['target'].str.isnumeric()]['target']

47        233990+
97        331990+
112       201990+
124       494800+
130       590000+
           ...   
376909    327990+
376966    234990+
376972    231100+
376976    1900/mo
377002    433500+
Name: target, Length: 7655, dtype: object

In [408]:
# уберем знак плюса из строки с целевым показателем
# так как непонятно, что именно он может означать в датасете
df['target'] = df['target'].apply(lambda target: target.replace('+', ''))

In [409]:
# проверим строки в которых содержится подстрока \mo
df[df['target'].str.contains('/mo')]

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
547,for rent,NaN,single-family home,4323 N Central Park Ave,3.5 Baths,"{'atAGlanceFacts': [{'factValue': '1913', 'fac...",yes,Chicago,"[{'rating': ['1/10', '4/10', '2/10', 'None/10'...","3,300 sqft",60618,4 Beds,IL,NaN,NaN,NaN,10588057,5500/mo
609,for rent,NaN,multi-family,220 Boylston St #1412,2 Baths,"{'atAGlanceFacts': [{'factValue': '1985', 'fac...",yes,Boston,"[{'rating': [], 'data': {'Distance': [], 'Grad...","1,673 sqft",2116,2 Beds,MA,NaN,NaN,NaN,72580936,10500/mo
2075,for rent,NaN,single-family home,2830 NE 56th Ct,4 Baths,"{'atAGlanceFacts': [{'factValue': '1965', 'fac...",NaN,Fort Lauderdale,"[{'rating': ['6/10', '2/10', '4/10'], 'data': ...","2,400 sqft",33308,4 Beds,FL,NaN,NaN,yes,A10521855,6390/mo
3025,for rent,NaN,multi-family,411 Kline Aly,2.5 Baths,"{'atAGlanceFacts': [{'factValue': '2014', 'fac...",NaN,Clarksville,"[{'rating': ['8/10', '9/10', '7/10'], 'data': ...","1,280 sqft",37040,2 Beds,TN,NaN,NaN,NaN,2102821,1200/mo
3645,for rent,NaN,multi-family,240 E Illinois St #2011,2 Baths,"{'atAGlanceFacts': [{'factValue': '2003', 'fac...",NaN,Chicago,"[{'rating': ['4/10', '7/10'], 'data': {'Distan...","1,473 sqft",60611,2 Beds,IL,NaN,NaN,yes,10590275,3600/mo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371791,for rent,NaN,multi-family,9436 Turrentine Dr,1.5 Baths,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,El Paso,"[{'rating': ['4/10', '8/10', '6/10'], 'data': ...","1,050 sqft",79925,2 Beds,TX,NaN,NaN,NaN,820163,890/mo
372459,for rent,NaN,townhouse,34 Jonquil Pl,2.5 Baths,"{'atAGlanceFacts': [{'factValue': '2014', 'fac...",NaN,The Woodlands,"[{'rating': ['5/10', '8/10', '7/10', '8/10'], ...","2,601 sqft",77375,3 Beds,TX,NaN,NaN,NaN,62158637,2500/mo
374288,for rent,NaN,single-family home,8864 Devonshire Dr,2 Baths,"{'atAGlanceFacts': [{'factValue': '2016', 'fac...",yes,Fort Worth,"[{'rating': ['6/10', '5/10', '5/10'], 'data': ...","2,000 sqft",76131,4 Beds,TX,NaN,NaN,NaN,NaN,2000/mo
375550,for rent,NaN,townhouse,2217 W Seybert St,NaN,"{'atAGlanceFacts': [{'factValue': '1920', 'fac...",NaN,Philadelphia,"[{'rating': ['1/10', '3/10'], 'data': {'Distan...",720 sqft,19121,2 Beds,PA,NaN,NaN,NaN,PAPH857944,1500/mo


Судя по всему в данном датасете также содержится информация о собственности домов сдающихся в аренду. Но так как нам необходимо построить модель которая будет предсказывать стоимость домов, а не аренду, удалим данные строки из датасета

In [410]:
df = df.drop(df[df['target'].str.contains('/mo')].index)

In [348]:
# еще раз сделаем проверку, что в датасете не осталось строк, которые не являются числом
df[~df['target'].str.isnumeric()]['target']

Series([], Name: target, dtype: object)

In [349]:
# преобразуем столбец к числовому виду
df['target'] = df['target'].astype(float)

##### Преобразуем столбец со статусом собственности.

In [350]:
# посмотрим на значения в столбце
df['status'].value_counts()

for sale                   156054
Active                     105206
For sale                    43464
foreclosure                  5677
New construction             5458
                            ...  
Coming soon: Nov 8.             1
Coming soon: Oct 29.            1
Coming soon: Dec 15.            1
Pending W/Backup Wanted         1
Coming soon: Dec 23.            1
Name: status, Length: 155, dtype: int64

Похоже, что некоторые значения дублируются в разном регистре. Необходимо это учесть при преобразовании.

In [351]:
# посмотрим на значения в столбце, игнорируя регистр
df['status'].str.lower().value_counts()

for sale                  199520
active                    105206
foreclosure                 6020
new construction            5458
pending                     4802
                           ...  
coming soon: dec 15.           1
coming soon: dec 25.           1
pending backups wanted         1
coming soon: nov 23.           1
coming soon: dec 23.           1
Name: status, Length: 148, dtype: int64

Большая часть собственности имеет статус 'for sale' или 'active'. Оставим эти два статуса, а остальные сгруппируем в один.

In [352]:
def group_status(status: str):
    if (status is not np.NaN and status.lower() in ['for sale', 'active']):
        return status.lower()
    return 'other'

In [353]:
df['status'] = df['status'].apply(group_status)

In [354]:
# проверим итоговый результат преобразования столбца
df['status'].value_counts()

for sale    199520
active      105206
other        69531
Name: status, dtype: int64

##### Преобразуем столбец с информацией о наличии бассейна

Датасет содержит два столбца с информацией о наличии бассейна. Cоздадим новый столбец на основе двух существующих столбцов.

In [355]:
# проверим содержимое полей
print(df['private pool'].str.lower().value_counts())
print(df['PrivatePool'].str.lower().value_counts())

yes    4151
Name: private pool, dtype: int64
yes    40025
Name: PrivatePool, dtype: int64


In [356]:
# функция для определения наличия бассейна
def fill_pool_data(df: pd.DataFrame):
    if (df['PrivatePool'] is not np.NaN and df['PrivatePool'].lower() == 'yes'):
        return 1
    if (df['private pool'] is not np.NaN and df['private pool'].lower() == 'yes'):
        return 1
    return 0

In [357]:
df['private_pool'] = df.apply(fill_pool_data, axis=1)

In [358]:
# удаляем столбцы, которые больше не нужны
df = df.drop(['PrivatePool', 'private pool'], axis=1)

##### Преобразуем столбец с типом собственности.

In [359]:
# посмотрим на значения в столбце
df['propertyType'].value_counts()

single-family home                                             91153
Single Family                                                  61886
Single Family Home                                             31725
condo                                                          25874
lot/land                                                       20506
                                                               ...  
1 Story, Contemporary, Other (See Remarks)                         1
Custom, Elevated, Other                                            1
Contemporary, Farmhouse                                            1
2 Stories, Traditional, Mediterranean, Texas Hill Country          1
Bilevel, Converted Dwelling, Loft with Bedrooms, Condo/Unit        1
Name: propertyType, Length: 1279, dtype: int64

In [360]:
# посмотрим на 15 самых популярных значения
df['propertyType'].value_counts().head(15)

single-family home               91153
Single Family                    61886
Single Family Home               31725
condo                            25874
lot/land                         20506
Condo                            16494
townhouse                        11388
Land                             10933
multi-family                      7742
Condo/Townhome/Row Home/Co-Op     7701
Townhouse                         6909
Traditional                       5912
coop                              3264
Multi Family                      2771
High Rise                         1823
Name: propertyType, dtype: int64

Похоже часть значений дублируется с разным написанием. Сгруппируем значения самых популярных типов.

In [361]:
def rename_property_type(type: str):
    if (type is np.NaN):
        return 'other'
    type = type.lower()
    if ('single' in type and ('family' in type or 'detached' in type)):
        return 'single-family home'
    if ('multi' in type and 'family' in type):
        return 'multi-family home'
    if ('lot' in type or 'land' in type):
        return 'land'
    return type

In [362]:
df['propertyType'] = df['propertyType'].apply(rename_property_type)

In [363]:
# посмотрим на результат группировки
df['propertyType'].value_counts().head(15)

single-family home               188094
condo                             42368
other                             34785
land                              31473
townhouse                         18297
multi-family home                 12037
condo/townhome/row home/co-op      7701
traditional                        5912
coop                               3264
high rise                          1823
ranch                              1781
detached, one story                1614
mobile/manufactured                1613
contemporary                       1556
1 story                            1235
Name: propertyType, dtype: int64

Оставим 5 самых популярных значения остальные сгруппируем в одно

In [364]:
def group_property_type(type: str):
    if (type in ['single-family home', 'condo', 'land', 'townhouse', 'multi-family home']):
        return type
    return 'other'

In [365]:
df['propertyType'] = df['propertyType'].apply(group_property_type)

In [366]:
# посмотрим на результат
df['propertyType'].value_counts()

single-family home    188094
other                  81988
condo                  42368
land                   31473
townhouse              18297
multi-family home      12037
Name: propertyType, dtype: int64

##### Преобразуем столбец с улицей

In [367]:
# проверим количество уникальных улиц
df['street'].value_counts().head(15)

Address Not Disclosed         672
Undisclosed Address           516
(undisclosed Address)         391
Address Not Available         175
Unknown Address                72
2103 E State Hwy 21            57
11305 Gulf Fwy                 54
17030 Youngblood Rd.           38
NE 58th Cir                    34
9470 Lancaster Rd. SW          32
1 Palmer Dr                    27
8426 Terrace Valley Circle     25
9845 Basil Western Rd NW       25
6320 SW 89th Court Road        24
8447 SW 99th Street Rd         22
Name: street, dtype: int64

В столбце слишком много уникальных значений. Скорее всего из-за того, что кроме улицы, значение также содержит номер дома. Уберем номер дома, а так же сгруппируем неисвезтные адреса в одно значение.

In [368]:
def rename_street(street: str):
    
    if street is np.NaN:
        return 'unknown'
    
    street = street.lower()
    
    if ('undisclosed' in street \
        or 'not disclosed' in street \
        or 'not available' in street \
        or 'unknown' in street):
        
        return 'unknown'
    
    splited_street = street.split()
    
    if splited_street[0].isnumeric():
        return ' '.join(splited_street[1:])
    
    return street

In [369]:
# проверим сколько остается значений после применения функции
df['street'].apply(rename_street).value_counts()

unknown                 1837
riverside dr              84
w 24th st                 78
main st                   76
glasgow dr                72
                        ... 
algonquin rd               1
collins ave apt 3101       1
stone house dr             1
township road 106          1
pereida st                 1
Name: street, Length: 189704, dtype: int64

Уникальных значений в столбце даже после удаления номера дома все равно слишком много. Удалим столбец.

In [370]:
df = df.drop('street', axis=1)

##### Преобразуем столбец с количеством ванных комнат

In [371]:
# проверим уникальные значения в столбце
df['baths'].value_counts()

2 Baths          51945
3 Baths          35353
2                20429
2.0              16354
4 Baths          14711
                 ...  
32                   1
5.25 Baths           1
41.0                 1
Bathrooms: 21        1
44.0                 1
Name: baths, Length: 225, dtype: int64

Преобразуем значения в числовой тип.

In [372]:
def rename_bath(bath: str):
    if bath is np.NaN:
        return np.NaN

    bath = bath.replace(',', '')
    
    match = re.search('[0-9]+\.?[0-9]*', bath)
    if match:
        bath = match.group()
    else:
        bath = ''
    
    if bath == '':
        return '0'
    
    return bath.strip()

In [373]:
df['baths'] = df['baths'].apply(rename_bath)

In [374]:
df['baths'] = df['baths'].astype(float)

##### Проведем преобразование данных о доме.

In [375]:
# посмотрим, что хранится в этом признаке
df['homeFacts'].head()

0    {'atAGlanceFacts': [{'factValue': '2019', 'fac...
1    {'atAGlanceFacts': [{'factValue': '2019', 'fac...
2    {'atAGlanceFacts': [{'factValue': '1961', 'fac...
3    {'atAGlanceFacts': [{'factValue': '2006', 'fac...
4    {'atAGlanceFacts': [{'factValue': '', 'factLab...
Name: homeFacts, dtype: object

Похоже, что в столбце хранится словарь с данными о доме. Посмотрим какие данные можно найти в данном словаре.

In [376]:
keys = set()

def get_keyset_from_dict(dict_string: str):
    d = ast.literal_eval(dict_string)
    keys.update(d.keys())
    
df['homeFacts'].apply(get_keyset_from_dict)

print(keys)

{'atAGlanceFacts'}


Словарь содержит только один ключ, в котором хранится массив данных, которые тоже представляют собой словарь. Проверим какие типы данных встречаются у домов

In [377]:
labels = set()

def get_labels_from_home_facts(dict_string: str):
    d = ast.literal_eval(dict_string)
    facts = d['atAGlanceFacts']
    for fact in facts:
        labels.add(fact['factLabel'])
    
df['homeFacts'].apply(get_labels_from_home_facts)

print(labels)

{'lotsize', 'Heating', 'Remodeled year', 'Year built', 'Parking', 'Cooling', 'Price/sqft'}


Итак в словаре хранятся данные о годе последней реконструкции дома, паркинге, стоимости квадратного фута, обогреве, охлаждении, размере участка и годе постройки. Извлечем эти данные в отдельные столбцы.

In [378]:
def transform_home_facts(dict_string: str):
    new_home_facts = dict()
    d = ast.literal_eval(dict_string)
    facts = d['atAGlanceFacts']
    for fact in facts:
        new_home_facts[fact['factLabel']] = fact['factValue']
    
    return new_home_facts

In [379]:
# преобразуем структуры, чтобы было легчи извлекать значения
df['transformed_home_facts'] = df['homeFacts'].apply(transform_home_facts)

In [380]:
# разобъем словарь на отдельные признаки
df['remodeled_year'] = df['transformed_home_facts'].apply(lambda d: d['Remodeled year'])
df['parking'] = df['transformed_home_facts'].apply(lambda d: d['Parking'])
df['price_for_sqft'] = df['transformed_home_facts'].apply(lambda d: d['Price/sqft'])
df['heating'] = df['transformed_home_facts'].apply(lambda d: d['Heating'])
df['lot_size'] = df['transformed_home_facts'].apply(lambda d: d['lotsize'])
df['cooling'] = df['transformed_home_facts'].apply(lambda d: d['Cooling'])
df['year_built'] = df['transformed_home_facts'].apply(lambda d: d['Year built'])

In [381]:
# уберем ненужные теперь столбцы
df = df.drop(['homeFacts', 'transformed_home_facts'], axis=1)

##### Преобразуем столбец с данными о наличии камина

In [382]:
# посмотрим на уникальные значения в столбце
df['fireplace'].value_counts()

yes                                                                     49922
Yes                                                                     20619
1                                                                       14533
2                                                                        2432
Not Applicable                                                           1990
                                                                        ...  
Gas, Wood Burning, Two, Propane Logs Convey                                 1
Free-standing, Insert, Wood                                                 1
Wood Burning, Attached Fireplace Doors/Screen, Electric, Gas Starter        1
One, Living Room                                                            1
Ceiling Fan, SMAPL, Utility Connection, Walk-In Closets                     1
Name: fireplace, Length: 1652, dtype: int64

Похоже, что в поле содержится разнородная информация. Где-то указано просто наличие камина, где-то количество, где-то описание. Попробуем вытащить информацию о наличии камина.

In [383]:
def get_fireplace_number(fireplace: str):
    if fireplace is np.NaN:
        return 0
    
    fireplace_keywords = ['yes', 'fireplace', 'one', 
                          '1', 'two', '2', 'three', '3', 'gas', 
                          'electric logs', '4', '4+', '5', '6',
                          '7', '8', '9', 'wood', 'frplc', 'electric']
    fireplace = fireplace.lower()
    if fireplace == 'not applicable' or fireplace == '0' or fireplace == 'no':
        return 0
    for keyword in fireplace_keywords:
        if keyword in fireplace:
            return 1

    return 0

##### Преобразуем столбец с данными о городах

In [385]:
df['city'].str.lower().value_counts()

houston          24387
san antonio      15496
miami            15389
jacksonville      9907
dallas            8751
                 ...  
langley              1
newport beach        1
sherman vlg          1
lily dale            1
blue springs         1
Name: city, Length: 1904, dtype: int64

In [389]:
# колонка содержит очень много уникальных значений
# посмотрим на процентное распределение городов
(df['city'].str.lower().value_counts() / df.shape[0] * 100).head(10)

houston         6.516111
san antonio     4.140470
miami           4.111880
jacksonville    2.647111
dallas          2.338233
brooklyn        1.934767
orlando         1.859952
new york        1.844989
chicago         1.834034
charlotte       1.741317
Name: city, dtype: float64

In [393]:
(df['city'].str.lower().value_counts() / df.shape[0] * 100).head(255).sum()

88.24604483015682

При дальнейшем кодировании, при использовании бинарного кодирования можно будет с помощью 8 новых столбцов описать 88 процентов данных.

##### Преобразуем столбец с данными о школах

In [122]:
# посмотрим на значения в столбце
df['schools']

0         [{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...
1         [{'rating': ['4/10', 'None/10', '4/10'], 'data...
2         [{'rating': ['8/10', '4/10', '8/10'], 'data': ...
3         [{'rating': ['9/10', '9/10', '10/10', '9/10'],...
4         [{'rating': ['4/10', '5/10', '5/10'], 'data': ...
                                ...                        
377180    [{'rating': ['10/10', '5/10'], 'data': {'Dista...
377181    [{'rating': ['1/10', '5/10', '7/10'], 'data': ...
377182    [{'rating': ['5/10', '4/10'], 'data': {'Distan...
377183    [{'rating': ['NA', 'NA', 'NA'], 'data': {'Dist...
377184    [{'rating': ['5/10', '4/10', '3/10'], 'data': ...
Name: schools, Length: 374257, dtype: object

Похоже, что поле содержит сложную структуру данных, представляющую собой массив со словарями. Попробуем извлечь данные из структуры.

In [123]:
df['schools'] = df['schools'].apply(lambda schools: ast.literal_eval(schools))
df['schools_rating'] = df['schools'].apply(lambda schools: schools[0]['rating'])
df['schools_distance'] = df['schools'].apply(lambda schools: schools[0]['data']['Distance'])
df['schools_grades'] = df['schools'].apply(lambda schools: schools[0]['data']['Grades'])
df['schools_name'] = df['schools'].apply(lambda schools: schools[0]['name'])

In [395]:
df = df.drop('schools', axis=1)

KeyError: "['schools'] not found in axis"

##### Преобразуем столбец с данными о площади

In [396]:
df['sqft'].value_counts()

0                                          11853
1,200 sqft                                   824
1,000 sqft                                   643
1,100 sqft                                   566
1,800 sqft                                   558
                                           ...  
101,415 sqft                                   1
3938                                           1
Total interior livable area: 4,580 sqft        1
32,552 sqft                                    1
Total interior livable area: 4,615 sqft        1
Name: sqft, Length: 25369, dtype: int64

In [397]:
def transform_sqft(sqft: str):
    if sqft is np.NaN:
        return np.NaN
    if type(sqft) is float:
        return sqft
    
    sqft = sqft.replace(',', '')
    match = re.search('[0-9]+\.?[0-9]*', sqft)
    if match:
        sqft = match.group()
    
    return sqft
    

In [398]:
df['sqft'] = df['sqft'].apply(transform_sqft)

##### Преобразуем столбец с данными о количестве спальных мест

In [128]:
df['beds'].value_counts()

3 Beds        52942
4 Beds        35143
3             31190
2 Beds        26077
4             19915
              ...  
8.93 acres        1
5,510 sqft        1
3.8 acres         1
7,104 sqft        1
8,479 sqft        1
Name: beds, Length: 1146, dtype: int64

In [412]:
def transform_beds(beds):
    if beds is np.NaN:
        return beds
    
    if 'acres' in beds or 'sqft' in beds:
        return np.NaN
    
    return beds

In [ ]:
df['beds'].apply()

##### Преобразуем столбец с количеством этажей

In [142]:
def get_stories_number(stories: str):
    if stories is np.NaN:
        return stories
    
    if type(stories) is float:
        return stories
    
    if stories.replace('.', '').isnumeric():
        return float(stories)
    
    stories = stories.lower()
    
    if 'one' in stories or '1' in stories:
        return 1.0
    
    if 'two' in stories or '2' in stories:
        return 2.0
    
    if 'three' in stories or '3' in stories:
        return 3.0
    
    if 'four' in stories or '4' in stories:
        return 4.0
    
    if 'five' in stories or '5' in stories:
        return 5.0
    
    if 'six' in stories or '6' in stories:
        return 6.0
    
    # в случае если значение в строке не удается преобразовать возвращаем пустое значение,
    # чтобы в дальнейшем его заменить
    return np.NaN 

In [143]:
df['stories'] = df['stories'].apply(get_stories_number)